<a href="https://colab.research.google.com/github/fevzican1/Project/blob/main/Metin_%C3%96zetleme_Modeli_(Sorunsuz_Calisan_Versiyon).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Gerekli kütüphaneleri içe aktarıyoruz
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import os
import sys

# NLTK 'punkt' ve 'punkt_tab' veri setlerini indirdiğimizden emin oluyoruz
# Bu veri setleri, `sent_tokenize` fonksiyonu için gereklidir.
print("NLTK 'punkt' ve 'punkt_tab' veri setleri indiriliyor...")
# Bu komutlar, veri setlerini otomatik olarak indirir ve kurar.
# Hata oluşmasını engellemek için doğrudan indirme komutlarını çalıştırıyoruz.
try:
    nltk.download('punkt', quiet=True)
    nltk.download('punkt_tab', quiet=True) # 'punkt_tab' da indiriliyor
    print("NLTK 'punkt' ve 'punkt_tab' veri setleri başarıyla indirildi veya zaten yüklü.")
except Exception as e:
    # İndirme işlemi başarısız olursa, kullanıcıya bilgi veriyoruz.
    print(f"Hata: Gerekli NLTK veri setleri indirilemedi. Lütfen internet bağlantınızı kontrol edin. Hata mesajı: {e}")
    sys.exit("Gerekli NLTK veri setleri indirilemedi. Program sonlandırılıyor.")


# ==============================================================================
# ÖRNEK METİN:
# Kendi metninizi buraya yapıştırabilirsiniz.
# Modelin daha iyi çalışması için uzun bir metin olması önerilir.
# ==============================================================================
long_text = """
Yapay zeka, modern teknolojinin en heyecan verici ve en hızlı gelişen alanlarından biridir. Makine öğrenimi ve derin öğrenme, yapay zekanın temelini oluşturan iki önemli alt daldır. Bu teknolojiler, algoritmaların büyük veri setleri üzerinde öğrenmesini ve insan benzeri kararlar almasını sağlar. Günümüzde yapay zeka, otonom araçlardan sağlık sektöründeki teşhislere, finansal piyasalardaki tahminlerden doğal dil işlemeye kadar birçok alanda kullanılmaktadır. Ancak, yapay zeka etiği ve güvenlik gibi konular da bu gelişmelerle birlikte önemli tartışmaları beraberinde getirmektedir.

Yapay zekanın geleceği, sinir ağları ve transformatör mimarileri gibi yeniliklerle şekillenmektedir. Özellikle doğal dil işleme (NLP) alanında, BERT ve GPT gibi modeller devrim niteliğinde başarılar elde etmiştir. Bu modeller, sadece metni anlamakla kalmaz, aynı zamanda metin üretebilir, çeviri yapabilir ve karmaşık soruları yanıtlayabilir. Bu gelişmeler, insan-bilgisayar etkileşimini kökten değiştirme potansiyeline sahiptir.

Sağlık alanında yapay zeka, hastalıkların erken teşhisinde, kişiselleştirilmiş tedavi planları oluşturmada ve ilaç keşfi süreçlerini hızlandırmada büyük faydalar sağlamaktadır. Görüntü işleme teknolojileri, kanserli hücrelerin tespitinde radyologlara yardımcı olmaktadır. Finans sektöründe ise yapay zeka, dolandırıcılık tespiti ve piyasa trendlerinin analizi gibi konularda önemli bir rol oynamaktadır.
"""

# ==============================================================================
# KOD BAŞLANGICI
# ==============================================================================

def extractive_summarizer(text, num_sentences=3):
    """
    Verilen metni özetleyici (extractive) yöntemle özetler.
    Metni cümlelere ayırır, her cümlenin vektörünü oluşturur ve en önemli cümleleri seçer.

    Args:
        text (str): Özetlenecek uzun metin.
        num_sentences (int): Özette yer alacak cümle sayısı.

    Returns:
        str: Oluşturulan özet metin.
    """
    # 1. Metni cümlelere ayırıyoruz
    sentences = sent_tokenize(text)

    if len(sentences) <= num_sentences:
        return text

    # 2. Önceden eğitilmiş bir model kullanarak cümle vektörlerini oluşturuyoruz
    # 'sentence-transformers' kütüphanesi otomatik olarak indirilecektir.
    print("\nCümle vektörleri oluşturuluyor. Bu işlem biraz zaman alabilir...")
    model_name = 'sentence-transformers/distilbert-base-nli-stsb-mean-tokens'
    model = SentenceTransformer(model_name)
    sentence_embeddings = model.encode(sentences)

    # 3. Cümleler arasındaki benzerliği hesaplamak için kosinüs benzerliği (cosine similarity) matrisini oluşturuyoruz
    similarity_matrix = cosine_similarity(sentence_embeddings, sentence_embeddings)

    # 4. Cümleleri sıralamak için TextRank benzeri bir algoritma kullanıyoruz
    # Bir NetworkX grafı oluşturuyoruz ve kenarları benzerlik skorlarına göre belirliyoruz
    graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(graph)

    # 5. En yüksek skora sahip cümleleri seçiyoruz
    ranked_sentences = sorted(((scores[i], sentence) for i, sentence in enumerate(sentences)), reverse=True)

    # 6. Seçilen cümleleri orijinal sırasına göre birleştirerek özeti oluşturuyoruz
    summary_sentences = []
    for i in range(num_sentences):
        summary_sentences.append(ranked_sentences[i][1])

    # Orijinal sırayı korumak için, cümlelerin orijinal metindeki sırasına göre sıralıyoruz
    summary_sentences.sort(key=lambda x: text.find(x))

    summary = ' '.join(summary_sentences)

    return summary

# Ana fonksiyonu çalıştırıyoruz
print("==============================================================================")
print("METİN ÖZETLEME ARACI (Özetleyici Yaklaşım)")
print("==============================================================================")
print("Orijinal Metin:")
print(long_text)
print("-" * 80)

# Özeti oluşturuyoruz
summary = extractive_summarizer(long_text, num_sentences=3)

print("\nOluşturulan Özet:")
print(summary)
print("=" * 80)

NLTK 'punkt' ve 'punkt_tab' veri setleri indiriliyor...
NLTK 'punkt' ve 'punkt_tab' veri setleri başarıyla indirildi veya zaten yüklü.
METİN ÖZETLEME ARACI (Özetleyici Yaklaşım)
Orijinal Metin:

Yapay zeka, modern teknolojinin en heyecan verici ve en hızlı gelişen alanlarından biridir. Makine öğrenimi ve derin öğrenme, yapay zekanın temelini oluşturan iki önemli alt daldır. Bu teknolojiler, algoritmaların büyük veri setleri üzerinde öğrenmesini ve insan benzeri kararlar almasını sağlar. Günümüzde yapay zeka, otonom araçlardan sağlık sektöründeki teşhislere, finansal piyasalardaki tahminlerden doğal dil işlemeye kadar birçok alanda kullanılmaktadır. Ancak, yapay zeka etiği ve güvenlik gibi konular da bu gelişmelerle birlikte önemli tartışmaları beraberinde getirmektedir.

Yapay zekanın geleceği, sinir ağları ve transformatör mimarileri gibi yeniliklerle şekillenmektedir. Özellikle doğal dil işleme (NLP) alanında, BERT ve GPT gibi modeller devrim niteliğinde başarılar elde etmiştir. Bu m

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


Oluşturulan Özet:
Günümüzde yapay zeka, otonom araçlardan sağlık sektöründeki teşhislere, finansal piyasalardaki tahminlerden doğal dil işlemeye kadar birçok alanda kullanılmaktadır. Ancak, yapay zeka etiği ve güvenlik gibi konular da bu gelişmelerle birlikte önemli tartışmaları beraberinde getirmektedir. Sağlık alanında yapay zeka, hastalıkların erken teşhisinde, kişiselleştirilmiş tedavi planları oluşturmada ve ilaç keşfi süreçlerini hızlandırmada büyük faydalar sağlamaktadır.
